In [ ]:
#Using Spacy

In [1]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")


In [3]:
# Acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}


def replace_acronyms(text, acronym_map):
    doc = nlp(text)  # Process the input text using spaCy

    replaced_text = []
    for token in doc:
        # If the token is an acronym, replace it with its full form from acronym_map
        if token.text in acronym_map:
            # Replace with full form, checking context if needed (you can expand this)
            replaced_text.append(acronym_map[token.text])
        else:
            replaced_text.append(token.text)

    return " ".join(replaced_text)

# Example usage
text = "The CD for the project is approaching, and the GP is discussing the IP extension with the LI."
replaced_text = replace_acronyms(text, acronym_map)
print(replaced_text)

The Closing Date for the project is approaching , and the General Partner is discussing the Investment Period extension with the LI.These all write in Closing Date storage


In [ ]:
#Using regular expression

In [10]:
import re

# Define the acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}

# Function to replace acronyms using regular expressions
def replace_acronyms(text, acronym_map):
    # Replace acronyms in the text
    for acronym, full_form in acronym_map.items():
        regex_pattern = r'\b' + re.escape(acronym) + r'\b'
        text = re.sub(regex_pattern, full_form, text)

    return text

# Example text input
input_text = "The CD is set, and the GP along with LI invested in the IF during the IP. This entire information available in CD memory"

# Replace acronyms in the text
output_text = replace_acronyms(input_text, acronym_map)

print(output_text)


The Closing Date is set, and the General Partner along with Lead Investor invested in the Investee Fund during the Investment Period. This entire information available in Closing Date memory


In [ ]:
#Using spacy and re

In [23]:
import re
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Define the acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}

def replace_acronyms(text, acronym_map):
    doc = nlp(text)
    sentences = list(doc.sents)

    # Prepare a mapping of replacements
    replacements = {}

    for sentence in sentences:
        # Extract tokens from the sentence
        tokens = [token.text for token in sentence]

        for acronym, full_form in acronym_map.items():
            # If the acronym is in the sentence, check context
            if acronym in tokens:
                # Collect context words around the acronym
                index = tokens.index(acronym)
                before_context = tokens[max(0, index-5):index]
                after_context = tokens[index+1:index+6]

                # Check if the context suggests the acronym's full form
                before_text = ' '.join(before_context).lower()
                after_text = ' '.join(after_context).lower()

                if any(word in full_form.lower() for word in before_text.split() + after_text.split()):
                    replacements[acronym] = full_form

    # Replace acronyms in the text
    for acronym, full_form in replacements.items():
        text = re.sub(r'\b{}\b'.format(re.escape(acronym)), full_form, text)

    return text



# Example text input
input_text = "The CD is set, and the GP along with LI invested in the IF during the IP. This entire information available in CD memory."


output_text = replace_acronyms(input_text, acronym_map)

print("Output text:")
print(output_text)


Output text:
The Closing Date is set, and the GP along with Lead Investor invested in the Investee Fund during the Investment Period. This entire information available in Closing Date memory.


In [29]:
#Using fuzzy wuzzy

#!pip install fuzzywuzzy python-Levenshtein



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.9 MB/s eta 0:00:00


In [33]:
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}

def replace_acronyms(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]

    # Iterate over tokens and match with acronyms
    new_tokens = []
    for token in tokens:
        # Find best match from acronym_map
        match, score = process.extractOne(token, acronym_map.keys(), scorer=fuzz.partial_ratio)
        if score > 80:  # Adjust threshold as needed
            # Replace with the full term if there's a good match
            new_tokens.append(acronym_map[match])
        else:
            new_tokens.append(token)

    return ' '.join(new_tokens)

# Example usage
text = "The CD for the DI is set for next week, and the GP needs to review the RF and DI."
print(replace_acronyms(text))


The Closing Date for the Direct Investment is set for next week , and the General Partner needs to review the Related Funds and Direct Investment .
